In [21]:
class Fn:
    
    def __init__(self, fn):
        self.fn = fn

    def __get__(self, instance, owner):
        print(instance, owner)

In [22]:
def dec(fn):
    return Fn(fn)

In [23]:
class Hi:
    @dec
    def my_fn(self):
        return

In [24]:
Hi.my_fn

None <class '__main__.Hi'>


In [18]:
h = Hi()

In [20]:
Hi.fn

None <class '__main__.Hi'>


In [19]:
h.fn

<__main__.Hi object at 0x11793e5f8> <class '__main__.Hi'>


# `metadsl`

...is a framework for creating domain specific languages in Python.

In [4]:
from metadsl import *
from metadsl_core import *
from metadsl_visualize import *

Here, we show a NumPy API compatible DSL that we have started writing. We can create a symbolic expression and then execute it. The widget shows the different stages of execution:

In [6]:
N = 100
expr = (arange(N) + arange(N))[5].to_ndarray()
expr

Typez(definitions=None, nodes={'7389996727730567975': ['32', PrimitiveNode(type='int', repr='100')], '-413479252060675646': ['7', CallNode(function='arange', type_params=None, args=['7389996727730567975'], kwargs=None)], '733488207619754122': ['8', CallNode(function='NDArrayCompat.__add__', type_params=None, args=['-413479252060675646', '-413479252060675646'], kwargs=None)], '7389996727835572900': ['25', PrimitiveNode(type='int', repr='5')], '939638188172311092': ['4', CallNode(function='NDArrayCompat.__getitem__', type_params=None, args=['733488207619754122', '7389996727835572900'], kwargs=None)], '9164808068233093317': ['5', CallNode(function='NDArrayCompat.to_ndarray', type_params=None, args=['939638188172311092'], kwargs=None)], '-5513958925783040194': ['9', CallNode(function='Converter.convert', type_params={'T': DeclaredTypeInstance(type='NDArray', params=None)}, args=['733488207619754122'], kwargs=None)], '-4362328496569185312': ['11', CallNode(function='Converter.convert', type

TypeError: 'PlaceholderExpression' object is not subscriptable

metadsl.expressions.PlaceholderExpression[numpy.ndarray](NDArrayCompat.to_ndarray, [<class 'metadsl_core.numpy.NDArrayCompat'>(NDArrayCompat.__getitem__, [<class 'metadsl_core.numpy.NDArrayCompat'>(NDArrayCompat.__add__, [<class 'metadsl_core.numpy.NDArrayCompat'>(arange, [100], {}), <class 'metadsl_core.numpy.NDArrayCompat'>(arange, [100], {})], {}), 5], {})], {})

Now, we can add an optimization replacement, so that we do the getitem before adding the two expressions:

In [7]:
@register_optimize
@rule
def optimize_getitem_add(l: NDArray, r: NDArray, idx: IndxType) -> R[NDArray]:
    return (
        # expression to match against
        (l + r)[idx],
        # expression to replace it with
        l[idx] + r[idx]
    )

Now, when we look at the execution steps, we see that it is doing each getitem before adding, which should be more efficient:

In [8]:
expr

Typez(definitions=None, nodes={'7389996727730567975': ['32', PrimitiveNode(type='int', repr='100')], '-413479252060675646': ['7', CallNode(function='arange', type_params=None, args=['7389996727730567975'], kwargs=None)], '733488207619754122': ['8', CallNode(function='NDArrayCompat.__add__', type_params=None, args=['-413479252060675646', '-413479252060675646'], kwargs=None)], '7389996727835572900': ['25', PrimitiveNode(type='int', repr='5')], '939638188172311092': ['4', CallNode(function='NDArrayCompat.__getitem__', type_params=None, args=['733488207619754122', '7389996727835572900'], kwargs=None)], '9164808068233093317': ['5', CallNode(function='NDArrayCompat.to_ndarray', type_params=None, args=['939638188172311092'], kwargs=None)], '-5513958925783040194': ['9', CallNode(function='Converter.convert', type_params={'T': DeclaredTypeInstance(type='NDArray', params=None)}, args=['733488207619754122'], kwargs=None)], '-4362328496569185312': ['11', CallNode(function='Converter.convert', type

TypeError: 'PlaceholderExpression' object is not subscriptable

metadsl.expressions.PlaceholderExpression[numpy.ndarray](NDArrayCompat.to_ndarray, [<class 'metadsl_core.numpy.NDArrayCompat'>(NDArrayCompat.__getitem__, [<class 'metadsl_core.numpy.NDArrayCompat'>(NDArrayCompat.__add__, [<class 'metadsl_core.numpy.NDArrayCompat'>(arange, [100], {}), <class 'metadsl_core.numpy.NDArrayCompat'>(arange, [100], {})], {}), 5], {})], {})

This shows how we can break up the NumPy API into different layers, all of which are extensible:

1. A compatibility layer that works like the existing NumPy API, except isn't limited to the Python types of the current API
2. A type safe version of this API. The conversion between the compatability layer and this layer is extensible, so that third party authors can add new conversion between their own Python objects and the typed representation.
3. A backend layer that translates either back to Python calls or source code, or to other targets like LLVM or Tensorflow.

The key is that all these layers are composable, so you could have different frontends for any of them or add your own. This is all done through a typed replacement system that is compatible with static analysis using MyPy.

`metadsl` is the glue that allows us to define this API in a way that enables greater greater collaboration. Since each layer has it's own well defined specification, different projects can extend and target these seperate layers, while minimizing the need to explicitly opt in to collaborate. The goal is to enable greater reuse, interoperability, and extensability within the Python scientific ecosystem.